# **1.순환 신경망(Recurrent Neural Network)**
* 시계열 또는 자연어와 같은 Sequence 데이터를 모델링하는 데 강력한 신경망. 시계열 데이터나 시퀀스 데이터를 잘 처리
* 예) 주식 가격, 텍스트 데이터, 오디오 데이터
* Sequence: 단어의 문장. 연결되어 있는 정보
<center><img src='https://velog.velcdn.com/images/softwarerbfl/post/d6c16b45-ef7d-4f3c-9a4a-efa54d2de587/image.png' with=400></center>

### 1-1. RNN동작 방식
* 은닉층에 노드에서 활성화 함수를 통해 나온 결과값을 다시 출력층 방향으로 보내면서 은닉층 노드의 다음 계산의 입력으로 보내는 것이 특징
* 셀(cell): 은닉층에서 활성화 함수를 통해 나온 결과를 내보내는 역할을 하는 것. 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행
* 은닉 상태(hidden state): 셀이 출력층 방향 또는 다음 시점인 t+1의 자신에게 보내는 값

```
rnn = torch.nn.RNN(input_size, hidden_size)
outputs, state = rnn(input_data)
# state: hidden state
```

### 1-2. inupt size
* 단어가 입력되면 각 글자를 백터의 형태로 변환하여 ont-hot encoding 해주는 과정이 필요
* "hello"
    * h = [1, 0, 0, 0]
    * e = [0, 1, 0 , 0]
    * l = [0, 0, 1, 0]
    * o = [0, 0, 0, 1]
* input size를 4
* input data의 세번째 차원으로 입력

### 1-3. hidden state size
* hidden state의 size는 output의 세번째 차원
* output size와 같음
* 셀에서 연산된 결과를 두가지로 나눠 하나는 output으로 출력되고, 다른 하나는 hidden state로 다음 step에 그대로 저장하고 전해짐

### 1-4. Sequence Length
* Sequence가 총 몇 개인지를 나타냄
    * "hello"
        * x0 = [1, 0, 0, 0]
        * x1 = [0, 1, 0 , 0]
        * x2 = [0, 0, 1, 0]
        * x3 = [0, 0, 1, 0]
        * x4 = [0, 0, 0, 1]
        * => 'hello'를 입력으로 보내면 sequence length는 5
* "hello"를 입력으로 보내면 sequence length는 5
* PyTorch에서는 모델이 sequence length를 알아서 파악하기 때문에 파라미터로 전달해 줄 필요가 없음

### 1-5. Batch Size
* 여러 데이터를 묶어 하나의 batch로 만들어 학습을 진행
* h, e, l, o를 가지고 만들 수 있는 데이터([h, e, l, l, 0][e, o, l, l, l][l, l, e, e, l]처럼) 중 배치사이즈로 묶어 학습을 진행
* batch size를 모델에서 파악하고 output data, input data에서의 첫번째 차원에 위치함

In [ ]:
import torch
import numpy as np
from torch.nn import RNN

In [ ]:
input_size = 4  # 입력 데이터의 크기. 각 입력 벡터의 길이
hidden_size = 2  # 은닉 상태의 크기. RNN 셀 내부의 은닉 상태 벡터의 크기

In [ ]:
# 입력 데이터를 one-hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [ ]:
input_data_np = np.array([[h, e, l, l, o],
                          [e, o, l, l, l],
                          [l, l, e, e, l]], dtype = np.float32)
# numpy 배열을 PyTorch Tensor로 변환
input_data = torch.Tensor(input_data_np)

In [ ]:
# RNN 모델을 정의 (입력 크기와 은닉 상태 크기를 인자로)
rnn = RNN(input_size, hidden_size)  # 4, 2
# 입력 데이터를 RNN에 넣어 출력을 계산 => outputs와 status를 반환
outputs, state = rnn(input_data)

In [ ]:
# 출력 데이터
outputs

tensor([[[-0.0453, -0.4226],
         [ 0.6697, -0.7710],
         [-0.0649, -0.4790],
         [-0.0649, -0.4790],
         [ 0.7129, -0.4935]],

        [[ 0.7190, -0.6954],
         [ 0.4997,  0.0411],
         [ 0.0523, -0.3278],
         [ 0.0523, -0.3278],
         [-0.4470, -0.0427]],

        [[-0.4256,  0.0462],
         [-0.3929, -0.3382],
         [ 0.6772, -0.6975],
         [ 0.6772, -0.6975],
         [ 0.2442, -0.5855]]], grad_fn=<StackBackward0>)

In [ ]:
# 은닉 상태
state  # 5개의 결과 => input데이터가 5개라서 => [h, e, l, l, o] 5개

tensor([[[-0.4256,  0.0462],
         [-0.3929, -0.3382],
         [ 0.6772, -0.6975],
         [ 0.6772, -0.6975],
         [ 0.2442, -0.5855]]], grad_fn=<StackBackward0>)

In [ ]:
# test를 사용하여 고유한 문자 집합을 만들기
test = 'hello! world'
string_set = list(set(test))
print(string_set)

['o', 'r', 'e', ' ', 'd', 'h', 'w', '!', 'l']


In [ ]:
# 각 문자를 인덱스로 매핑하는 사전을 생성하기
string_dic = {c: i for i, c in enumerate(string_set)}
print(string_dic)  # 인덱스화 됨

{'o': 0, 'r': 1, 'e': 2, ' ': 3, 'd': 4, 'h': 5, 'w': 6, '!': 7, 'l': 8}


In [ ]:
# 이를 기반으로 입력 벡터와 은닉 상태 벡터의 크기를 설정
input_size = len(string_dic)
print(input_size)
hidden_size = len(string_dic)
print(hidden_size)

9
9


In [ ]:
# test라는 문자열을 string_dic이라는 사전에 기반하여 인덱스 리스트로 변환
test_idx = [string_dic[c] for c in test]
print(test_idx)  # 인덱스가 나옴

[5, 2, 8, 8, 0, 7, 3, 6, 0, 1, 8, 4]


In [ ]:
# test_idx 리스트를 x_data로 변환
x_data = [test_idx[:]]
print(x_data)  # 하나 더 감싸서 나옴

[[5, 2, 8, 8, 0, 7, 3, 6, 0, 1, 8, 4]]


```
[array([[0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]])]
```

In [ ]:
# x_data 리스트의 각 요소를 one-hot 인코딩 형식으로 변환하여 'x_one_hot '리스트를 만들기
x_one_hot = [np.eye(input_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.]])]


In [ ]:
y_data = [test_idx[:]]
print(y_data)

[[5, 2, 8, 8, 0, 7, 3, 6, 0, 1, 8, 4]]


In [ ]:
X = torch.FloatTensor(x_one_hot)
y = torch.LongTensor(y_data)

<ipython-input-15-25a499c695e1>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  X = torch.FloatTensor(x_one_hot)


In [ ]:
rnn = RNN(input_size, hidden_size)
loss_fun = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = 0.1)

RNN 모델 학습

In [ ]:
# outputs, state = rnn(X)
# loss: x, predition: x(idx), predition_str: s(str)
# hello! world
epochs = 100

for i in range(epochs):
    optimizer.zero_grad()
    outputs, state = rnn(X)

    # (배치사이즈, 시퀀스길이, 히든사이즈)
    # outputs: (1, 12, 12) -> (12, 12)
    # y: (1, 12) -> (12,)
    loss = loss_fun(outputs.view(-1, input_size), y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis = 2)
    result_str = ''.join(string_set[ch] for ch in np.squeeze(result))
    print(i, 'loss: ', loss)
    print('prediction: ', result, ' prediction_str: ', result_str)
    # 8부터 제대로 예측

0 loss:  tensor(2.1904, grad_fn=<NllLossBackward0>)
prediction:  [[2 2 7 7 0 6 0 8 0 0 7 2]]  prediction_str:  ee!!owoloo!e
1 loss:  tensor(1.9756, grad_fn=<NllLossBackward0>)
prediction:  [[0 2 8 8 0 8 0 8 0 0 8 2]]  prediction_str:  oellololoole
2 loss:  tensor(1.8089, grad_fn=<NllLossBackward0>)
prediction:  [[0 2 8 8 0 8 0 8 0 0 8 0]]  prediction_str:  oellololoolo
3 loss:  tensor(1.6759, grad_fn=<NllLossBackward0>)
prediction:  [[8 2 8 8 0 8 0 8 0 8 8 8]]  prediction_str:  lellolololll
4 loss:  tensor(1.5601, grad_fn=<NllLossBackward0>)
prediction:  [[8 2 8 8 0 8 3 8 0 8 8 8]]  prediction_str:  lellol lolll
5 loss:  tensor(1.4528, grad_fn=<NllLossBackward0>)
prediction:  [[8 2 8 8 0 8 3 8 0 1 8 4]]  prediction_str:  lellol lorld
6 loss:  tensor(1.3508, grad_fn=<NllLossBackward0>)
prediction:  [[5 2 8 8 0 7 3 8 0 1 8 4]]  prediction_str:  hello! lorld
7 loss:  tensor(1.2543, grad_fn=<NllLossBackward0>)
prediction:  [[5 2 8 8 0 7 3 8 0 1 8 4]]  prediction_str:  hello! lorld
8 loss: 

### RNN의 단점
* 입력과 출력이 고정
* 기울기 소실
* 단점을 극복하기 위해 RNN의 발전 형태인 LSTM과 GRU를 사용(문제를 완벽히 해결하지 못함)